In [21]:
import pandas as pd
import os
import gzip
import ftplib
from io import BytesIO, TextIOWrapper

def open_ftp():
  rootDir = ""
  result = ftplib.FTP("ftp.ncbi.nlm.nih.gov")
  result.login("anonymous", "quanhu.sheng.1@vumc.org")
  result.cwd(rootDir)
  return(result)

taxonomyFile = '/data/cqs/references/bacteria/20220406_taxonomy.txt'
assemblySummaryFile = '/data/cqs/references/bacteria/20220406_assembly_summary_refseq.txt'
taxonomyRootId = 10239 
outputFolder = '/data1/shengq2/references/spcount'
prefix = '20220406_virus'
referenceAndRepresentativeOnly = True
genomeNumberPerFile=500
ref_categories = ['reference genome', 'representative genome']


In [22]:

taxonomy=pd.read_csv(taxonomyFile, sep="\t", index_col=0)
taxonomy.head()

,ParentId,ScientificName,Rank,superkingdom,kingdom,phylum,class,order,family,genus,species
Id,,,,,,,,,,,
1,1,root,no rank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131567,Bacteria,superkingdom,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,335928,Azorhizobium,genus,2.0,NaN,1224.0,28211.0,356.0,335928.0,6.0,NaN
7,6,Azorhizobium caulinodans,species,2.0,NaN,1224.0,28211.0,356.0,335928.0,6.0,7.0
9,32199,Buchnera aphidicola,species,2.0,NaN,1224.0,1236.0,91347.0,1903409.0,32199.0,9.0


In [23]:
root=taxonomy.loc[taxonomyRootId]
print(root)
root_taxonomy=taxonomy[taxonomy[root.Rank]==taxonomyRootId]
root_taxonomy.head()



ParentId                     1
ScientificName         Viruses
Rank              superkingdom
superkingdom           10239.0
kingdom                    NaN
phylum                     NaN
class                      NaN
order                      NaN
family                     NaN
genus                      NaN
species                    NaN
Name: 10239, dtype: object


,ParentId,ScientificName,Rank,superkingdom,kingdom,phylum,class,order,family,genus,species
Id,,,,,,,,,,,
10239,1,Viruses,superkingdom,10239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10240,2732527,Poxviridae,family,10239.0,2732005.0,2732007.0,2732525.0,2732527.0,10240.0,NaN,NaN
10241,10240,Chordopoxvirinae,subfamily,10239.0,2732005.0,2732007.0,2732525.0,2732527.0,10240.0,NaN,NaN
10242,10241,Orthopoxvirus,genus,10239.0,2732005.0,2732007.0,2732525.0,2732527.0,10240.0,10242.0,NaN
10243,10242,Cowpox virus,species,10239.0,2732005.0,2732007.0,2732525.0,2732527.0,10240.0,10242.0,10243.0


In [24]:
localDir = outputFolder
if not os.path.exists(localDir):
  os.mkdir(localDir)

cacheDir = os.path.join(localDir, "cache")
if not os.path.exists(cacheDir):
  os.mkdir(cacheDir)

fastaDir = os.path.join(localDir, "fasta")
if not os.path.exists(fastaDir):
  os.mkdir(fastaDir)


In [25]:
assembly=pd.read_csv(assemblySummaryFile, sep="\t", header=1, index_col=0)
assembly.head()


/scratch/cqs/shengq2/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,...,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date
# assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_000001215.4,PRJNA164,SAMN02803731,NaN,reference genome,7227,7227,Drosophila melanogaster,NaN,NaN,latest,...,Full,2014/08/01,Release 6 plus ISO1 MT,The FlyBase Consortium/Berkeley Drosophila Gen...,GCA_000001215.4,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
GCF_000001405.39,PRJNA168,NaN,NaN,reference genome,9606,9606,Homo sapiens,NaN,NaN,latest,...,Full,2019/02/28,GRCh38.p13,Genome Reference Consortium,GCA_000001405.28,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
GCF_000001635.27,PRJNA169,NaN,NaN,reference genome,10090,10090,Mus musculus,NaN,NaN,latest,...,Full,2020/06/24,GRCm39,Genome Reference Consortium,GCA_000001635.9,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
GCF_000001735.4,PRJNA116,SAMN03081427,NaN,reference genome,3702,3702,Arabidopsis thaliana,ecotype=Columbia,NaN,latest,...,Full,2018/03/15,TAIR10.1,The Arabidopsis Information Resource (TAIR),GCA_000001735.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome,9785,9785,Loxodonta africana,NaN,ISIS603380,latest,...,Full,2009/07/15,Loxafr3.0,Broad Institute,GCA_000001905.1,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na


In [26]:
root_assembly=assembly[assembly.taxid.isin(root_taxonomy.index)]
root_assembly.head()

,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,...,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date
# assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_000819615.1,PRJNA485481,NaN,NaN,na,2886930,10847,Escherichia phage phiX174,NaN,NaN,latest,...,Full,1993/04/28,ViralProj14015,NaN,GCA_000819615.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,ICTV species exemplar,na
GCF_000820355.1,PRJNA485481,NaN,NaN,na,191289,191289,Sclerophthora macrospora virus A,NaN,NaN,latest,...,Full,2004/03/16,ViralMultiSegProj14361,NaN,GCA_000820355.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
GCF_000820495.2,PRJNA485481,NaN,NaN,na,518987,11520,Influenza B virus (B/Lee/1940),NaN,NaN,latest,...,Full,1993/08/02,ViralMultiSegProj14656,NaN,GCA_000820495.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,ICTV species exemplar,na
GCF_000836805.1,PRJNA485481,NaN,NaN,reference genome,90963,2003329,Chlamydia phage CPG1,strain=PhiCPG1,NaN,latest,...,Full,1999/01/02,ViralProj14012,NaN,GCA_000836805.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,ICTV species exemplar,na
GCF_000836825.1,PRJNA485481,NaN,NaN,na,1891754,1891754,Bos taurus polyomavirus 1,NaN,NaN,latest,...,Full,1993/04/29,ViralProj14017,NaN,GCA_000836825.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,ICTV species exemplar,na


In [27]:
root_assembly.refseq_category.value_counts()

na                  11801
reference genome       48
Name: refseq_category, dtype: int64

In [28]:

if referenceAndRepresentativeOnly:
  root_assembly=root_assembly.loc[root_assembly.refseq_category.isin(ref_categories)]
  print(root_assembly.shape)

#root_assembly.describe()


(48, 22)


In [29]:
print(root_assembly.columns)
print(root_assembly.shape)

Index(['bioproject', 'biosample', 'wgs_master', 'refseq_category', 'taxid',
       'species_taxid', 'organism_name', 'infraspecific_name', 'isolate',
       'version_status', 'assembly_level', 'release_type', 'genome_rep',
       'seq_rel_date', 'asm_name', 'submitter', 'gbrs_paired_asm',
       'paired_asm_comp', 'ftp_path', 'excluded_from_refseq',
       'relation_to_type_material', 'asm_not_live_date'],
      dtype='object')
(48, 22)


In [35]:
class AssemblyGenome:
  def __init__(self, accession, taxid, organism_name, ftp_path, local_dir):
    self.accession = accession
    self.taxid = taxid
    self.organism_name = organism_name
    self.ftp_path = ftp_path
    file_prefix=os.path.basename(ftp_path)
    file_name=file_prefix + "_genomic.fna.gz"
    self.remote_fna_path=os.path.join(ftp_path, file_name)
    self.url_file = self.remote_fna_path.replace("ftp://ftp.ncbi.nlm.nih.gov", "")
    self.url_file = self.url_file.replace("https://ftp.ncbi.nlm.nih.gov", "")
    remote_dir=os.path.dirname(self.url_file)
    sub_folder = os.path.join(local_dir, remote_dir[1:])
    if not os.path.exists(sub_folder):
      os.makedirs(sub_folder)
    self.local_fna_path=os.path.join(sub_folder, file_name)
    self.local_done_path=self.local_fna_path + ".done"

genome=AssemblyGenome('a', 1, 'a', 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/836/805/GCF_000836805.1_ViralProj14012', '/data1/shengq2/spcount')
print(genome.url_file)
print(genome.local_fna_path)

'/genomes/all/GCF/000/836/805/GCF_000836805.1_ViralProj14012/GCF_000836805.1_ViralProj14012_genomic.fna.gz'

'/genomes/all/GCF/000/836/805/GCF_000836805.1_ViralProj14012/GCF_000836805.1_ViralProj14012_genomic.fna.gz_genomic.fna.gz'

In [31]:
  
genomes = []
for row in root_assembly.itertuples():
  genome = AssemblyGenome(row.Index, row.taxid, row.organism_name, row.ftp_path, cacheDir )
  genomes.append(genome)

print(len(genomes))


48


In [34]:
genomes[0].ftp_path

'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/836/805/GCF_000836805.1_ViralProj14012'

In [130]:
#download all genome
totalCount = len([g for g in genomes if not os.path.exists(g.local_done_path)])
if totalCount > 0:
  with open_ftp() as ftp:
    currentCount = 0
    for genome in genomes:
      localFile = genome.local_fna_path
      localDoneFile = localFile + ".done"

      if os.path.exists(localDoneFile):
        continue

      currentCount = currentCount + 1

      remoteFile=genome.remote_fna_path.replace("https://ftp.ncbi.nlm.nih.gov", "")

      print("Downloading %d/%d: %s to %s ..." % (currentCount, totalCount, remoteFile, localFile))
      for retry in [1,2,3]:
        #time.sleep(0.1)
        try:
          with open(localFile, "wb") as f:
            ftp.retrbinary("RETR " + remoteFile, f.write, 1024)
            open(localDoneFile, 'wt').close()
        except:
          print(f"ERROR: downloading {remoteFile} retry {retry} failed.")
          ftp.close()
          ftp = open_ftp()



ERROR: downloading /genomes/all/GCF/000/016/425/GCF_000016425.1_ASM1642v1/GCF_000016425.1_ASM1642v1_genomic.fna.gz retry 2 failed.
ERROR: downloading /genomes/all/GCF/000/024/005/GCF_000024005.1_ASM2400v1/GCF_000024005.1_ASM2400v1_genomic.fna.gz retry 3 failed.
ERROR: downloading /genomes/all/GCF/000/092/925/GCF_000092925.1_ASM9292v1/GCF_000092925.1_ASM9292v1_genomic.fna.gz retry 2 failed.
ERROR: downloading /genomes/all/GCF/000/166/695/GCF_000166695.1_ASM16669v1/GCF_000166695.1_ASM16669v1_genomic.fna.gz retry 3 failed.
ERROR: downloading /genomes/all/GCF/000/194/605/GCF_000194605.1_ASM19460v1/GCF_000194605.1_ASM19460v1_genomic.fna.gz retry 3 failed.
ERROR: downloading /genomes/all/GCF/000/219/125/GCF_000219125.1_ASM21912v1/GCF_000219125.1_ASM21912v1_genomic.fna.gz retry 3 failed.
ERROR: downloading /genomes/all/GCF/000/242/335/GCF_000242335.1_ASM24233v3/GCF_000242335.1_ASM24233v3_genomic.fna.gz retry 1 failed.
ERROR: downloading /genomes/all/GCF/000/277/795/GCF_000277795.1_ASM27779v1/

In [ ]:
output_ranks = ['superkingdom', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

with open(os.path.join(outputFolder, file_prefix + ".db.txt"), "wt") as fdb:
  with open(os.path.join(outputFolder, file_prefix + ".taxonomy.txt"), "wt") as ftx:
    ftx.write("chrom\taccession\tscientific_name\ttaxid\trank\t%s\n" % "\t".join(output_ranks))
    findex = 0
    fFasta = None
    gindex = 0
    for genome in genomes:
      gtax = root_taxonomy.loc[genome.taxid]
      if gindex % genomeNumberPerFile == 0:
        if fFasta != None:
          fFasta.close()
        findex += 1
        cur_file = os.path.join(fastaDir, "%s.%03d.fa" % (file_prefix, findex))
        fFasta = open(cur_file, "wb")
        fdb.write(cur_file + "\n")
      gindex += 1
      with gzip.open(genome.local_fna_path, 'rb') as f:
        file_content = f.read()
        fFasta.write(file_content)
        with TextIOWrapper(BytesIO(file_content)) as fin:
          for line in fin:
            if line.startswith(">"):
              parts = line.rstrip().split(' ')
              chrom = parts[0][1:]
              ftx.write("%s\t%s\t%s\t%s\t%s" % (chrom, genome.accession, genome.organism_name, genome.taxid, gtax['Rank']))
              for rank in output_ranks:
                if pd.isnull(gtax[rank]):
                  ftx.write("\tUnclassified")
                else:
                  rtex=root_taxonomy.loc[gtax[rank]]
                  ftx.write(f"\t{rtex.ScientificName}")
              ftx.write("\n")
    fFasta.close()    

